## Correction

In [ ]:
typo =   # Detection result(in list) 

In [1]:
import numpy as np
import pandas as pd
from collections import Counter 
from nltk import edit_distance

def typo_classification(typo,correct):
    if (len(typo) > len(correct)):
        return 'insertion'
    elif (len(typo) < len(correct)):
        return 'deletion'
    else:
        typo_count = Counter(typo)
        correct_count = Counter(correct)
        if typo_count == correct_count:
            return 'reversal'
        else:
            return 'substitution'

def find_candidates(typo,corpus):
    candidates = []
    candi_type = []
    for word in corpus:
        ed = edit_distance(typo,word)
        word_type = typo_classification(typo,word)
        if ((ed == 1) | ((ed == 2) & (word_type == 'reversal'))):
            candidates.append(word)
            candi_type.append(word_type)
    return candidates,candi_type

def find_position(typo,candidates):
    position = []
    for corr in candidates:
        typo_type = typo_classification(typo,corr)
        
        if (typo_type == 'deletion'):
            typo += '#'

            i = 0
            while i < len(corr):
                if (corr[i] != typo[i]):
                    if corr[i] != corr[i-1]:
                        typo = typo[:-1]
                        position.append([typo,corr,"@",corr[i],i,typo_type])
                        break
                    else:
                        typo = typo[:-1]
                        position.append([typo,corr,"@",corr[i],i,typo_type])
                        position.append([typo,corr,"@",corr[i],i-1,typo_type])
                        break
                        
                i += 1
        elif (typo_type == 'insertion'):
            corr += '#'

            i = 0
            while i < len(corr):
                if (corr[i] != typo[i]):
                    if typo[i] != typo[i-1]:
                        corr = corr[:-1]
                        position.append([typo,corr,typo[i],"@",i,typo_type])
                        break
                    elif ((typo[i] == typo[i-1])& (typo[i] == typo[i-2])):
                        corr = corr[:-1]
                        position.append([typo,corr,typo[i],"@",i,typo_type])
                        position.append([typo,corr,typo[i],"@",i-1,typo_type])
                        position.append([typo,corr,typo[i],"@",i-2,typo_type])
                        break
                    else:
                        corr = corr[:-1]
                        position.append([typo,corr,typo[i],"@",i,typo_type])
                        position.append([typo,corr,typo[i],"@",i-1,typo_type])
                        break
                i += 1
        elif (typo_type == 'substitution'):
            i = 0
            while i < len(corr):
                if (corr[i] != typo[i]):
                    position.append([typo,corr,typo[i],corr[i],i,typo_type])
                    break
                i+=1
                
        elif (typo_type == 'reversal'):
            i = 0
            while i < len(corr)-1:
                if ((typo[i] == corr[i+1]) & (typo[i+1] == corr[i])):
                    typo_comb = typo[i] + typo[i+1]
                    position.append([typo,corr,typo_comb,typo_comb[::-1],i,typo_type])
                    break
                i +=1
    return position

In [ ]:
output = []

for typo in typo:
    
    candidates,cand_type = find_candidates(typo,corpus)
    correction = find_position(typo,candidates)
    correction = pd.DataFrame(correction)
    
    if correction.empty:  
        output.append(typo)
        
    else:
        correction.columns = ['Typo','Correction','old','new','index','type']
        
        # 1. calculate the prior
    
        freq = [] # the number of times that the proposed correction c appears in the training set
        for cor in correction['Correction']:
            freq.append(len(wrong[wrong[0] == cor]))    
    
        N = len(truth_clean) # the number of words
        V = len(set(truth_clean)) # the vocabulary size 

        prior = (pd.DataFrame(freq) + 0.5)/(N + V/2)
     
        correction['probability of c'] = prior

        # 2. Calculate the likelihood
        
    
    
        # 3. Calculate the posterior and find the correction that has maximum posterior
        
        correction['posterior'] = correction['probability of c'] * correction['probability of t given c']
        p = correction[correction['posterior'] == max(correction['posterior'])]
        maxp = p['Correction'][p['Correction'].index[0]]
        output.append(''.join(maxp))
        

In [ ]:
output